In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

import anndata
from ALLCools.clustering import significant_pc_test


In [2]:
# Parameters
group_name = "STR"


In [3]:
t = group_name
use_genes = pd.read_csv(f'use_genes.txt', header=None, index_col=0).index
use_genes


Index(['ENSMUSG00000025905', 'ENSMUSG00000033740', 'ENSMUSG00000067879',
       'ENSMUSG00000042501', 'ENSMUSG00000048960', 'ENSMUSG00000016918',
       'ENSMUSG00000025938', 'ENSMUSG00000025935', 'ENSMUSG00000025937',
       'ENSMUSG00000025932',
       ...
       'ENSMUSG00000054843', 'ENSMUSG00000025089', 'ENSMUSG00000025092',
       'ENSMUSG00000041362', 'ENSMUSG00000006270', 'ENSMUSG00000025094',
       'ENSMUSG00000087095', 'ENSMUSG00000043969', 'ENSMUSG00000057858',
       'ENSMUSG00000117790'],
      dtype='object', name=0, length=3402)

In [4]:
ref_adata = anndata.read_h5ad(glob.glob(f'{t}_*_rs1_mch.h5ad')[0])[:, use_genes]
ref_adata

View of AnnData object with n_obs × n_vars = 12546 × 3402
    obs: 'mCCCFrac', 'mCGFrac', 'mCHFrac', 'FinalmCReads', 'DissectionRegion', 'Plate', 'Col384', 'Row384', 'Slice', 'Sample', 'Technology', 'InputReads', 'PassBasicQC', 'PlateNormCov', 'CEMBARegion', 'MajorRegion', 'SubRegion', 'L1', 'L1_annot', 'L4', 'L4Region', 'L2_annot'
    var: 'chrom', 'cov_mean', 'end', 'start', 'L4_enriched_features'
    uns: 'L4_feature_enrichment', 'log'

In [5]:
np.random.seed(0)

n_train_cell = 100000
# select mC cells to fit the model
train_cell = np.zeros(ref_adata.shape[0]).astype(bool)
if ref_adata.shape[0] > n_train_cell:
    train_cell[np.random.choice(np.arange(ref_adata.shape[0]), n_train_cell,
                                False)] = True
else:
    train_cell[:] = True

ref_adata.obs['Train'] = train_cell.copy()
ref_adata.obs['Train'].sum()


12546

In [6]:
if (ref_adata.obs['Train'].sum() >= 100) and (ref_adata.shape[1]>=100):
    #model = PCA(n_components=100, svd_solver='arpack', random_state=0)
    #TruncatedSVD is for sparse matrices whereas PCA is for normal matrices and will center data
    # defore SVD
    model = TruncatedSVD(n_components=100, algorithm='arpack', random_state=0) 
else:
    model = TruncatedSVD(n_components=ref_adata.X[ref_adata.obs['Train'].values].shape[1]-1, svd_solver='arpack', random_state=0)
    #model = PCA(n_components=ref_adata.X[ref_adata.obs['Train'].values].shape[1]-1, svd_solver='arpack', random_state=0)

# use selected mC cells to fit
model.fit(ref_adata.X[ref_adata.obs['Train'].values])
sel_dim = (model.singular_values_ != 0)
print(sel_dim.sum())

100


In [7]:
## Transform RS1

chunk_size = 50000
chunks = []
for chunk_start in range(0, ref_adata.shape[0], chunk_size):
    chunks.append(
        model.transform(ref_adata.X[chunk_start:(chunk_start + chunk_size)]))

ref_adata.obsm['cef_pca'] = np.concatenate(chunks, axis=0)[:, sel_dim]
ref_adata.obsm['cef_pca'] /= model.singular_values_[sel_dim]


In [8]:
npc = significant_pc_test(ref_adata, p_cutoff=0.1, update=False, obsm='cef_pca')


23 components passed P cutoff of 0.1.


In [9]:
ref_adata.write_h5ad(f'{t}_{ref_adata.shape[0]}_rs1_mch.h5ad')


In [10]:
## Transform RNA

qry_adata = anndata.read_h5ad(glob.glob(f'{t}_*_rna.h5ad')[0])
chunks = []
for chunk_start in range(0, qry_adata.shape[0], chunk_size):
    # tmp = (qry_adata.X[chunk_start:(chunk_start + chunk_size)].toarray() - qry_adata.var['mean'].values) / qry_adata.var['std'].values
    tmp = (qry_adata.X[chunk_start:(chunk_start + chunk_size)])
    chunks.append(model.transform(tmp))
    print(chunk_start)

qry_adata.obsm['cef_pca'] = np.concatenate(chunks, axis=0)[:, sel_dim]
qry_adata.obsm['cef_pca'] /= model.singular_values_[sel_dim]


0


In [11]:
qry_adata.write_h5ad(f'{t}_{qry_adata.shape[0]}_rna.h5ad')


In [12]:
## Transform RS2

qry_adata = anndata.read_h5ad(glob.glob(f'{t}_*_rs2_mch.h5ad')[0])
chunks = []
for chunk_start in range(0, qry_adata.shape[0], chunk_size):
    chunks.append(
        model.transform(qry_adata.X[chunk_start:(chunk_start + chunk_size)]))

qry_adata.obsm['cef_pca'] = np.concatenate(chunks, axis=0)[:, sel_dim]
qry_adata.obsm['cef_pca'] /= model.singular_values_[sel_dim]


In [13]:
qry_adata.write_h5ad(f'{t}_{qry_adata.shape[0]}_rs2_mch.h5ad')
